
# 使用Turicreate进行音乐推荐



In [2]:
!pip install turicreate

     |████████████████████████████████| 33.1 MB 13 kB/s eta 0:00:0197   |████████▍                       | 8.6 MB 19 kB/s eta 0:20:35     |███████████████████▌            | 20.2 MB 24 kB/s eta 0:08:42     |███████████████████▊            | 20.4 MB 12 kB/s eta 0:16:49     |██████████████████████████████▉ | 31.8 MB 39 kB/s eta 0:00:32
     |████████████████████████████████| 3.5 MB 18 kB/s eta 0:00:0115   |███████▋                        | 829 kB 12 kB/s eta 0:03:30
  Using cached resampy-0.2.1.tar.gz (322 kB)
     |████████████████████████████████| 175.3 MB 8.8 kB/s ta 0:00:018   |██▏                             | 12.0 MB 62 kB/s eta 0:43:16     |████████▎                       | 45.2 MB 21 kB/s eta 1:39:12     |████████▎                       | 45.5 MB 17 kB/s eta 2:07:02     |█████████                       | 48.7 MB 36 kB/s eta 0:58:32     |█████████▍                      | 51.2 MB 40 kB/s eta 0:50:59     |██████████▍                     | 56.8 MB 14 kB/s eta 2:11:51     |████████████

     |████████████████████████████████| 147 kB 37 kB/s eta 0:00:01    |███████████████▋                | 71 kB 37 kB/s eta 0:00:03
     |████████████████████████████████| 77 kB 59 kB/s eta 0:00:01
  Created wheel for resampy: filename=resampy-0.2.1-py3-none-any.whl size=320848 sha256=13ea513477f71d513b03a44efe8db091cffc1adaab907123f0fa8d5babfdbeaf
  Stored in directory: /Users/datalab/Library/Caches/pip/wheels/71/74/53/d5ceb7c5ee7a168c7d106041863e71ac3273f4a4677743a284
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=f6234bda25caf8e0d32efe0478ea409a20565a1652f05830529936dabbebc345
  Stored in directory: /Users/datalab/Library/Caches/pip/wheels/3f/e3/ec/8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2
  Created wheel for absl-py: filename=absl_py-0.9.0-py3-none-any.whl size=121931 sha256=a0b4551addb6f776d7d3404e842ec5243b8894947bba6be78bc66c56963d37b5
  Stored in directory: /Users/datalab/Library/Caches/pip/wheels/cc/af/1a/498a24d0730ef484019e0

In [3]:
import turicreate as tc

 下载数据
http://s3.amazonaws.com/dato-datasets/millionsong/10000.txt


In [4]:
#train_file = 'http://s3.amazonaws.com/dato-datasets/millionsong/10000.txt'
train_file = '/Users/datalab/bigdata/cjc/millionsong/song_usage_10000.txt'
sf = tc.SFrame.read_csv(train_file, header=False, delimiter='\t', verbose=False)
sf = sf.rename({'X1':'user_id', 'X2':'music_id', 'X3':'rating'})

In [5]:
train_set, test_set = sf.random_split(0.8, seed=1)

In [6]:
popularity_model = tc.popularity_recommender.create(train_set, 
                                                    'user_id', 'music_id', 
                                                    target = 'rating')

Preparing data set.

Data has 1599753 observations with 76085 users and 10000 items.

Data prepared in: 4.15079s

1599753 observations to process; with 10000 unique items.

In [7]:
item_sim_model = tc.item_similarity_recommender.create(train_set, 
                                                       'user_id', 'music_id', 
                                                       target = 'rating', 
                                                       similarity_type='cosine')

Preparing data set.

Data has 1599753 observations with 76085 users and 10000 items.

Data prepared in: 3.7942s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 4.01s                               | 100              | 10000           |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 5.31028s

In [8]:
factorization_machine_model = tc.recommender.factorization_recommender.create(train_set, 
                                                                              'user_id', 'music_id',
                                                                              target='rating')

Preparing data set.

Data has 1599753 observations with 76085 users and 10000 items.

Data prepared in: 4.32575s

Training factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 8.8282

Final training RMSE: 2.96963

In [9]:
len(train_set)

1599753

In [10]:
result = tc.recommender.util.compare_models(test_set, 
                                            [popularity_model, item_sim_model, factorization_machine_model],
                                            user_sample=.5, skip_set=train_set)

compare_models: using 34354 users to estimate model performance
PROGRESS: Evaluate model M0


recommendations finished on 1000/34354 queries. users per second: 5393.6

recommendations finished on 2000/34354 queries. users per second: 5901.05

recommendations finished on 3000/34354 queries. users per second: 5891.65

recommendations finished on 4000/34354 queries. users per second: 5752.93

...

recommendations finished on 31000/34354 queries. users per second: 5997.94

recommendations finished on 32000/34354 queries. users per second: 5964.91

recommendations finished on 33000/34354 queries. users per second: 5996.43

recommendations finished on 34000/34354 queries. users per second: 6000.34


Precision and recall summary statistics by cutoff
+--------+-----------------------+------------------------+
| cutoff |     mean_precision    |      mean_recall       |
+--------+-----------------------+------------------------+
|   1    |  0.000611282528963149 | 0.00015967112604116908 |
|   2    | 0.0005094021074692894 | 0.00020969792246105614 |
|   3    | 0.0005142535561118539 | 0.0002897975387832757  |
|   4    | 0.0005312336263608328 | 0.0004549968288297451  |
|   5    | 0.0005414216685102181 | 0.0005703895459784241  |
|   6    |  0.000548213696609806 | 0.0007142050704628652  |
|   7    | 0.0005738570680062202 | 0.0008314573503625282  |
|   8    | 0.0005930895965535321 | 0.0009602957110903129  |
|   9    | 0.0006015796316780176 | 0.0011215295194248998  |
|   10   | 0.0005967281830354553 | 0.0012125256843862468  |
+--------+-----------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 7.641661679566184

Per User RMSE (best)
+------------------------------

In [12]:
K = 10
users = gl.SArray(sf['user_id'].unique().head(100))

In [13]:
recs = item_sim_model.recommend(users=users, k=K)
recs.head()

user_id,music_id,score,rank
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOXUQNR12AF72A69D6,3.022422651449839,1
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOUFAZA12AC3DFAB20,1.3368427753448486,2
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOSFSTC12A8C141219,1.091982126235962,3
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOVIWFP12A58A7D1BD,1.045163869857788,4
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOBMTQD12AB01833D0,1.0294516881306965,5
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOCMNRG12AB0189D3F,0.9756437937418619,6
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOXOHUM12A67ADC826,0.9506873289744059,7
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOWBFVW12A6D4F612B,0.9092370669047037,8
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOXFYTY127E9433E7D,0.8977278073628744,9
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOYBLYP12A58A79D32,0.8970928192138672,10
